In [1]:
from IPython.display import HTML

# Cell visibility - COMPLETE:
#tag = HTML('''<style>
#div.input {
#    display:none;
#}
#</style>''')
#display(tag)

#Cell visibility - TOGGLE:
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<p style="text-align:right">
Promijeni vidljivost <a href="javascript:code_toggle()">ovdje</a>.</p>''')
display(tag)

## Brza Fourierova transformacija (FFT)
Brza Fourierova transformacija (FFT) je algoritam koji izračunava diskretnu Fourierovu transformaciju (DFT) niza ili njezin inverz (IDFT). Fourierova analiza pretvara signal iz izvorne domene (najčešće vremenska ili prostorna) u prikaz u frekvencijskoj domeni i obrnuto.

U ovom primjeru moguće je analizirati (grafički) transformaciju signala iz vremenske domene u frekvencijsku domenu. Svojstva tri signala možete postaviti određivanjem odgovarajućih parametara za odabrane funkcije. Te će se funkcije onda automatski vizualizirati na prva dva grafa - kao tri različita individualna signala i kao njihova kumulativna kombinacija. Treći graf, prikazan na samom dnu ovog primjera, predstavlja FFT rezultat. Pored navedenoga, moguće je provjeriti i učinak apliciranja šuma.

U ovom se primjeru mogu koristiti i međusobno kombinirati sljedeće funkcije:
* sinusni val,
* kosinusni val,
* prigušeni val,
* rampa-funkcija,
* step-funkcija.

In [2]:
%matplotlib inline
#%config InlineBackend.close_figures=False 
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import Latex, display, Markdown # For displaying Markdown and LaTeX code
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.patches as mpatches
from IPython.display import HTML, clear_output
from IPython.display import display
from IPython.display import HTML
from ipywidgets import interactive, interactive_output, VBox, HBox
from ipywidgets import widgets
from scipy import pi
from scipy.fftpack import fft
from scipy import signal

### SLIDER WIDGETS
# Sine widgets
slider_a_1 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_f_1 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_p_1 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Cosine widgets
slider_acos_1 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fcos_1 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pcos_1 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Damping widgets
slider_adamp_1 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fdamp_1 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pdamp_1 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_d_1 = widgets.FloatSlider(description='Prigušenje', min=0., max=3., step=0.2, continuous_update=False)

# Sine widgets
slider_a_2 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_f_2 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_p_2 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Cosine widgets
slider_acos_2 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fcos_2 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pcos_2 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Damping widgets
slider_adamp_2 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fdamp_2 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pdamp_2 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_d_2 = widgets.FloatSlider(description='Prigušenje', min=0., max=3., step=0.2, continuous_update=False)

# Sine widgets
slider_a_3 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_f_3 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_p_3 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Cosine widgets
slider_acos_3 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fcos_3 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pcos_3 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)

# Damping widgets
slider_adamp_3 = widgets.FloatSlider(description='Amplituda', min=0., max=4., step=0.25, continuous_update=False)
slider_fdamp_3 = widgets.FloatSlider(description='Frekvencija', min=0., max=30., step=0.5, continuous_update=False)
slider_pdamp_3 = widgets.FloatSlider(description='Faza', min=-10.0, max=10.0, step=0.5, continuous_update=False)
slider_d_3 = widgets.FloatSlider(description='Prigušenje', min=0., max=3., step=0.2, continuous_update=False)

# Ramp widgets
slider_aramp_1 = widgets.FloatSlider(description='Parametar a', value = 0.0, min=0.0, max=2., step=0.25, continuous_update=False)
slider_aramp_2 = widgets.FloatSlider(description='Parametar a', value = 0.0, min=0.0, max=2., step=0.25, continuous_update=False)
slider_aramp_3 = widgets.FloatSlider(description='Parametar a', value = 0.0, min=0.0, max=2., step=0.25, continuous_update=False)

# Step widgets
slider_astep_1 = widgets.FloatSlider(description='Parametar a', value = 0., min=0, max=2, step=0.1, continuous_update=False)
slider_bstep_1 = widgets.FloatSlider(description='Parametar b', value = 1, min=0, max=4., step=0.25, continuous_update=False)

# Step widgets
slider_astep_2 = widgets.FloatSlider(description='Parametar a', value = 0., min=0, max=2, step=0.1, continuous_update=False)
slider_bstep_2 = widgets.FloatSlider(description='Parametar b', value = 1, min=0, max=4., step=0.25, continuous_update=False)

# Step widgets
slider_astep_3 = widgets.FloatSlider(description='Parametar a', value = 0., min=0, max=2, step=0.1, continuous_update=False)
slider_bstep_3 = widgets.FloatSlider(description='Parametar b', value = 1, min=0, max=4., step=0.25, continuous_update=False)

# Parameters
sample_rate = 1024
N = (2 - 0) * sample_rate
time = np.linspace(0, 2, N)
noise = np.random.normal (0, 0.5, N)
frequency = np.linspace (0.0, 512, int (N/2))
waves = [0, 0, 0]

In [3]:
# Layouts
fun1_layout = widgets.Layout(border='solid blue', width = '33%', height = '400', padding='1px')
fun2_layout = widgets.Layout(border='solid green', width = '33%', height = '400', padding='1px')
fun3_layout = widgets.Layout(border='solid red', width = '33%', height = '400', padding='1px')
outputs_layout = widgets.Layout(border='solid black', width = '100%', height = '200', padding='5px')

# Dropdown widgets
dd_fun1 = widgets.Dropdown(
    options=['Sinusni val', 'Kosinusni val', 'Prigušeni val', 'Rampa funkcija', 'Step funkcija'],
    value='Sinusni val',
    description='Funkcija:',
    disabled=False,
    style = {'description_width': 'initial'},
)

dd_fun2 = widgets.Dropdown(
    options=['Sinusni val', 'Kosinusni val', 'Prigušeni val', 'Rampa funkcija', 'Step funkcija'],
    value='Sinusni val',
    description='Funkcija:',
    disabled=False,
    style = {'description_width': 'initial'},
)

dd_fun3 = widgets.Dropdown(
    options=['Sinusni val', 'Kosinusni val', 'Prigušeni val', 'Rampa funkcija', 'Step funkcija'],
    value='Sinusni val',
    description='Funkcija:',
    disabled=False,
    style = {'description_width': 'initial'},
)

    
# Ploting
def plot_everything():
    global waves
    wave1 = np.array(waves[0])
    wave2 = np.array(waves[1])
    wave3 = np.array(waves[2])
    
    output_time1.clear_output(wait=True)
    output_time2.clear_output(wait=True)
    output_fft.clear_output(wait=True)
    
    # Plot 1: single functions
    with output_time1:
        #output_time1.clear_output()
        plt.figure(figsize=(10,5))
        plt.ylim(-5, 5)
        plt.plot (time, wave1, label="Signal 1", color="b")
        plt.plot (time, wave2, label="Signal 2", color="g")
        plt.plot (time, wave3, label="Signal 3", color="r")
        plt.title('Vremenska domena: tri odabrana signala')
        plt.xlabel ('Vrijeme [s]')
        plt.ylabel ('Amplituda')
        plt.grid(True)
        plt.axhline(y=0,lw=0.8,color='k')
        plt.axvline(x=0,lw=0.8,color='k')
        plt.legend(loc="upper right")
        plt.show()
       
    # Plot 2: combined wave
    add_noise = noise_widget.value
    if add_noise == True:
        time_data = wave1 + wave2 + wave3 + noise
    else:
         time_data = wave1 + wave2 + wave3 
    
    with output_time2:
        #output_time2.clear_output()
        plt.figure(figsize=(10,5))
        plt.ylim(-5, 5)
        plt.plot(time, time_data)
        plt.title('Vremenska domena: združeni signal')
        plt.xlabel('Vrijeme [s]')
        plt.ylabel('Amplituda')
        plt.grid(True)
        plt.axhline(y=0,lw=0.8,color='k')
        plt.axvline(x=0,lw=0.8,color='k')
        plt.show()
        display(noise_widget)
    
    # Plot 3: FFT
    freq_data = fft(time_data)
    y = 2/N * np.abs (freq_data [0:np.int (N/2)])
    
    with output_fft:
        #output_fft.clear_output()
        plt.figure(figsize=(10,5))
        plt.ylim(0, 5)
        plt.xlim(0, 40)
        plt.plot(frequency, y)
        plt.title('Frekvencijska domena')
        plt.xlabel('Frekvencija [Hz]')
        plt.ylabel('Magnituda')
        plt.grid(True)
        plt.axhline(y=0,lw=0.8,color='k')
        plt.axvline(x=0,lw=0.8,color='k')
        plt.show()


# Initial view
def first():
    global waves
    frequency = 0
    phase = 0
    A = 0
    function1 = [A * np.sin(2 * pi * x * frequency + phase) for x in time]
    function2 = [A * np.sin(2 * pi * x * frequency + phase) for x in time]
    function3 = [A * np.sin(2 * pi * x * frequency + phase) for x in time]
    waves = [function1, function2, function3]
    
    with output_fun1:
        display(slider_a_1, slider_f_1, slider_p_1)
    
    with output_fun2:
        display(slider_a_2, slider_f_2, slider_p_2)
        
    with output_fun3:
        display(slider_a_3, slider_f_3, slider_p_3)
    
    f_sine(1, slider_a_1.value, slider_f_1.value, slider_p_1.value)
    

#Function data
def f_sine(caller, A, frequency, phase):
    global waves  
    function1 = [A * np.sin(2 * pi * x *frequency + phase) for x in time]
    if caller == 1:
        waves[0] = function1
    if caller == 2:
        waves[1] = function1
    if caller == 3:
        waves[2] = function1
    plot_everything()

def f_cos(caller, A, frequency, phase):
    global waves
    function2 = [A * np.cos(2 * pi * x * frequency + phase) for x in time]
    if caller == 1:
        waves[0] = function2
    if caller == 2:
        waves[1] = function2
    if caller == 3:
        waves[2] = function2
    plot_everything()

def f_dwave(caller, A, frequency, phase, decay):
    global waves
    function3 = [A * math.exp(-decay * x) *(np.cos(2 * pi * x * frequency + phase)) for x in time]
    if caller == 1:
        waves[0] = function3
    if caller == 2:
        waves[1] = function3
    if caller == 3:
        waves[2] = function3
    plot_everything()
    
def f_ramp(caller, A):
    global waves
    step = lambda x, a: x - a if x > a else 0
    function4 = [step(t, A) for t in time]
    if caller == 1:
        waves[0] = function4
    if caller == 2:
        waves[1] = function4
    if caller == 3:
        waves[2] = function4
    plot_everything()
    
def f_step(caller, a, b):
    global waves
    step = lambda x, a, b: b if x > a else 0
    function5 = [step(t, a, b) for t in time]
    if caller == 1:
        waves[0] = function5
    if caller == 2:
        waves[1] = function5
    if caller == 3:
        waves[2] = function5
    plot_everything()

    
# slider observers        
def slider_change(change, sender):
    if sender == 'sin_a_1':
        f_sine(1, change['new'], slider_f_1.value, slider_p_1.value)
    if sender == 'sin_f_1':
        f_sine(1, slider_a_1.value, change['new'], slider_p_1.value)
    if sender == 'sin_p_1':
        f_sine(1, slider_a_1.value, slider_f_1.value, change['new'])
    
    if sender == 'cos_a_1':
        f_cos(1, change['new'], slider_fcos_1.value, slider_pcos_1.value)
    if sender == 'cos_f_1':
        f_cos(1, slider_acos_1.value, change['new'], slider_pcos_1.value)
    if sender == 'cos_p_1':
        f_cos(1, slider_acos_1.value, slider_fcos_1.value, change['new'])
        
    if sender == 'damp_a_1':
        f_dwave(1, change['new'], slider_fdamp_1.value, slider_pdamp_1.value, slider_d_1.value)
    if sender == 'damp_f_1':
        f_dwave(1, slider_adamp_1.value, change['new'], slider_pdamp_1.value, slider_d_1.value)
    if sender == 'damp_p_1':
        f_dwave(1, slider_adamp_1.value, slider_fdamp_1.value, change['new'], slider_d_1.value)
    if sender == 'damp_d_1':
        f_dwave(1, slider_adamp_1.value, slider_fdamp_1.value, slider_pdamp_1.value, change['new'])
        
    if sender == 'ramp_a_1':
        f_ramp(1, change['new'])
        
    if sender == 'step_a_1':
        f_step(1, change['new'], slider_bstep_1.value)
    if sender == 'step_b_1':
        f_step(1, slider_astep_1.value, change['new'])
    ###
    if sender == 'sin_a_2':
        f_sine(2, change['new'], slider_f_2.value, slider_p_2.value)
    if sender == 'sin_f_2':
        f_sine(2, slider_a_2.value, change['new'], slider_p_2.value)
    if sender == 'sin_p_2':
        f_sine(2, slider_a_2.value, slider_f_2.value, change['new'])
    
    if sender == 'cos_a_2':
        f_cos(2, change['new'], slider_fcos_2.value, slider_pcos_2.value)
    if sender == 'cos_f_2':
        f_cos(2, slider_acos_2.value, change['new'], slider_pcos_2.value)
    if sender == 'cos_p_2':
        f_cos(2, slider_acos_2.value, slider_fcos_2.value, change['new'])
        
    if sender == 'damp_a_2':
        f_dwave(2, change['new'], slider_fdamp_2.value, slider_pdamp_2.value, slider_d_2.value)
    if sender == 'damp_f_2':
        f_dwave(2, slider_adamp_2.value, change['new'], slider_pdamp_2.value, slider_d_2.value)
    if sender == 'damp_p_2':
        f_dwave(2, slider_adamp_2.value, slider_fdamp_2.value, change['new'], slider_d_2.value)
    if sender == 'damp_d_2':
        f_dwave(2, slider_adamp_2.value, slider_fdamp_2.value, slider_pdamp_2.value, change['new'])
        
    if sender == 'ramp_a_2':
        f_ramp(2, change['new'])
        
    if sender == 'step_a_2':
        f_step(2, change['new'], slider_bstep_2.value)
    if sender == 'step_b_2':
        f_step(2, slider_astep_2.value, change['new'])
    ###
    if sender == 'sin_a_3':
        f_sine(3, change['new'], slider_f_3.value, slider_p_3.value)
    if sender == 'sin_f_3':
        f_sine(3, slider_a_3.value, change['new'], slider_p_3.value)
    if sender == 'sin_p_3':
        f_sine(3, slider_a_3.value, slider_f_3.value, change['new'])
    
    if sender == 'cos_a_3':
        f_cos(3, change['new'], slider_fcos_3.value, slider_pcos_3.value)
    if sender == 'cos_f_3':
        f_cos(3, slider_acos_3.value, change['new'], slider_pcos_3.value)
    if sender == 'cos_p_3':
        f_cos(3, slider_acos_3.value, slider_fcos_3.value, change['new'])
        
    if sender == 'damp_a_3':
        f_dwave(3, change['new'], slider_fdamp_3.value, slider_pdamp_3.value, slider_d_3.value)
    if sender == 'damp_f_3':
        f_dwave(3, slider_adamp_3.value, change['new'], slider_pdamp_3.value, slider_d_3.value)
    if sender == 'damp_p_3':
        f_dwave(3, slider_adamp_3.value, slider_fdamp_3.value, change['new'], slider_d_3.value)
    if sender == 'damp_d_3':
        f_dwave(3, slider_adamp_3.value, slider_fdamp_3.value, slider_pdamp_3.value, change['new'])
        
    if sender == 'ramp_a_3':
        f_ramp(3, change['new'])
        
    if sender == 'step_a_3':
        f_step(3, change['new'], slider_bstep_3.value)
    if sender == 'step_b_3':
        f_step(3, slider_astep_3.value, change['new'])

slider_a_1.observe(lambda change: slider_change(change, 'sin_a_1'), names='value')
slider_f_1.observe(lambda change: slider_change(change, 'sin_f_1'), names='value')
slider_p_1.observe(lambda change: slider_change(change, 'sin_p_1'), names='value')

slider_acos_1.observe(lambda change: slider_change(change, 'cos_a_1'), names='value')
slider_fcos_1.observe(lambda change: slider_change(change, 'cos_f_1'), names='value')
slider_pcos_1.observe(lambda change: slider_change(change, 'cos_p_1'), names='value')

slider_adamp_1.observe(lambda change: slider_change(change, 'damp_a_1'), names='value')
slider_fdamp_1.observe(lambda change: slider_change(change, 'damp_f_1'), names='value')
slider_pdamp_1.observe(lambda change: slider_change(change, 'damp_p_1'), names='value')
slider_d_1.observe(lambda change: slider_change(change, 'damp_d_1'), names='value')

slider_aramp_1.observe(lambda change: slider_change(change, 'ramp_a_1'), names='value')

slider_astep_1.observe(lambda change: slider_change(change, 'step_a_1'), names='value')
slider_bstep_1.observe(lambda change: slider_change(change, 'step_b_1'), names='value')
###    
slider_a_2.observe(lambda change: slider_change(change, 'sin_a_2'), names='value')
slider_f_2.observe(lambda change: slider_change(change, 'sin_f_2'), names='value')
slider_p_2.observe(lambda change: slider_change(change, 'sin_p_2'), names='value')

slider_acos_2.observe(lambda change: slider_change(change, 'cos_a_2'), names='value')
slider_fcos_2.observe(lambda change: slider_change(change, 'cos_f_2'), names='value')
slider_pcos_2.observe(lambda change: slider_change(change, 'cos_p_2'), names='value')

slider_adamp_2.observe(lambda change: slider_change(change, 'damp_a_2'), names='value')
slider_fdamp_2.observe(lambda change: slider_change(change, 'damp_f_2'), names='value')
slider_pdamp_2.observe(lambda change: slider_change(change, 'damp_p_2'), names='value')
slider_d_2.observe(lambda change: slider_change(change, 'damp_d_2'), names='value')

slider_aramp_2.observe(lambda change: slider_change(change, 'ramp_a_2'), names='value')

slider_astep_2.observe(lambda change: slider_change(change, 'step_a_2'), names='value')
slider_bstep_2.observe(lambda change: slider_change(change, 'step_b_2'), names='value')
###    
slider_a_3.observe(lambda change: slider_change(change, 'sin_a_3'), names='value')
slider_f_3.observe(lambda change: slider_change(change, 'sin_f_3'), names='value')
slider_p_3.observe(lambda change: slider_change(change, 'sin_p_3'), names='value')

slider_acos_3.observe(lambda change: slider_change(change, 'cos_a_3'), names='value')
slider_fcos_3.observe(lambda change: slider_change(change, 'cos_f_3'), names='value')
slider_pcos_3.observe(lambda change: slider_change(change, 'cos_p_3'), names='value')

slider_adamp_3.observe(lambda change: slider_change(change, 'damp_a_3'), names='value')
slider_fdamp_3.observe(lambda change: slider_change(change, 'damp_f_3'), names='value')
slider_pdamp_3.observe(lambda change: slider_change(change, 'damp_p_3'), names='value')
slider_d_3.observe(lambda change: slider_change(change, 'damp_d_3'), names='value')

slider_aramp_3.observe(lambda change: slider_change(change, 'ramp_a_3'), names='value')

slider_astep_3.observe(lambda change: slider_change(change, 'step_a_3'), names='value')
slider_bstep_3.observe(lambda change: slider_change(change, 'step_b_3'), names='value')


# dropdown(1) selection change
def fun1_dropdown(change):
    if (dd_fun1.value == 'Sinusni val'): 
        with output_fun1:
            output_fun1.clear_output(wait=True)
            display(slider_a_1, slider_f_1, slider_p_1)
        f_sine(1, slider_a_1.value, slider_f_1.value, slider_p_1.value)    
    
    if (dd_fun1.value == 'Kosinusni val'):
        with output_fun1:
            output_fun1.clear_output(wait=True)
            display(slider_acos_1, slider_fcos_1, slider_pcos_1)
        f_cos(1, slider_acos_1.value, slider_fcos_1.value, slider_pcos_1.value)   
        
    if (dd_fun1.value == 'Prigušeni val'):
        with output_fun1:
            output_fun1.clear_output(wait=True)
            display(slider_adamp_1, slider_fdamp_1, slider_pdamp_1, slider_d_1)
        f_dwave(1, slider_adamp_1.value, slider_fdamp_1.value, slider_pdamp_1.value, slider_d_1.value) 

    if (dd_fun1.value == 'Rampa funkcija'):
        with output_fun1:
            output_fun1.clear_output(wait=True)
            display(slider_aramp_1)
        f_ramp(1, slider_aramp_1.value)   
               
    if (dd_fun1.value == 'Step funkcija'):
        with output_fun1:
            output_fun1.clear_output(wait=True)
            display(slider_astep_1, slider_bstep_1)
        f_step(1, slider_astep_1.value, slider_bstep_1.value) 

        
# dropdown(2) selection change
def fun2_dropdown(change):
    if (dd_fun2.value == 'Sinusni val'): 
        with output_fun2:
            output_fun2.clear_output(wait=True)
            display(slider_a_2, slider_f_2, slider_p_2)
        f_sine(2, slider_a_2.value, slider_f_2.value, slider_p_2.value)    
    
    if (dd_fun2.value == 'Kosinusni val'):
        with output_fun2:
            output_fun2.clear_output(wait=True)
            display(slider_acos_2, slider_fcos_2, slider_pcos_2)
        f_cos(2, slider_acos_2.value, slider_fcos_2.value, slider_pcos_2.value)   
        
    if (dd_fun2.value == 'Prigušeni val'):
        with output_fun2:
            output_fun2.clear_output(wait=True)
            display(slider_adamp_2, slider_fdamp_2, slider_pdamp_2, slider_d_2)
        f_dwave(2, slider_adamp_2.value, slider_fdamp_2.value, slider_pdamp_2.value, slider_d_2.value) 

    if (dd_fun2.value == 'Rampa funkcija'):
        with output_fun2:
            output_fun2.clear_output(wait=True)
            display(slider_aramp_2)
        f_ramp(2, slider_aramp_2.value)   
               
    if (dd_fun2.value == 'Step funkcija'):
        with output_fun2:
            output_fun2.clear_output(wait=True)
            display(slider_astep_2, slider_bstep_2)
        f_step(2, slider_astep_2.value, slider_bstep_2.value) 
         
            
# dropdown(3) selection change
def fun3_dropdown(change):
    if (dd_fun3.value == 'Sinusni val'): 
        with output_fun3:
            output_fun3.clear_output(wait=True)
            display(slider_a_3, slider_f_3, slider_p_3)
        f_sine(3, slider_a_3.value, slider_f_3.value, slider_p_3.value)    
    
    if (dd_fun3.value == 'Kosinusni val'):
        with output_fun3:
            output_fun3.clear_output(wait=True)
            display(slider_acos_3, slider_fcos_3, slider_pcos_3)
        f_cos(3, slider_acos_3.value, slider_fcos_3.value, slider_pcos_3.value)   
        
    if (dd_fun3.value == 'Prigušeni val'):
        with output_fun3:
            output_fun3.clear_output(wait=True)
            display(slider_adamp_3, slider_fdamp_3, slider_pdamp_3, slider_d_3)
        f_dwave(3, slider_adamp_3.value, slider_fdamp_3.value, slider_pdamp_3.value, slider_d_3.value) 

    if (dd_fun3.value == 'Rampa funkcija'):
        with output_fun3:
            output_fun3.clear_output(wait=True)
            display(slider_aramp_3)
        f_ramp(3, slider_aramp_3.value)   
               
    if (dd_fun3.value == 'Step funkcija'):
        with output_fun3:
            output_fun3.clear_output(wait=True)
            display(slider_astep_3, slider_bstep_3)
        f_step(3, slider_astep_3.value, slider_bstep_3.value) 
            
# dropdown observers   
dd_fun1.observe(fun1_dropdown, names = 'value')
dd_fun2.observe(fun2_dropdown, names = 'value')
dd_fun3.observe(fun3_dropdown, names = 'value')

# checkbox widget
def trigger(b):
    plot_everything()
noise_widget = widgets.Checkbox(False, description='Apliciraj šum')
noise_widget.observe(trigger)


# output layout
output_fun1 = widgets.Output()
output_fun2 = widgets.Output()
output_fun3 = widgets.Output()
output_time1 = widgets.Output(layout = outputs_layout)
output_time2 = widgets.Output(layout = outputs_layout)
output_fft = widgets.Output(layout = outputs_layout)

box1 = widgets.VBox([dd_fun1, output_fun1], layout = fun1_layout)
box2 = widgets.VBox([dd_fun2, output_fun2], layout = fun2_layout)
box3 = widgets.VBox([dd_fun3, output_fun3], layout = fun3_layout)
panel_box = widgets.HBox([box1,  widgets.Label(" "), box2, widgets.Label(" "), box3])
display(panel_box, output_time1, output_time2, output_fft)
first()



Output(layout=Layout(border='solid black', height='200', padding='5px', width='100%'))

Output(layout=Layout(border='solid black', height='200', padding='5px', width='100%'))

Output(layout=Layout(border='solid black', height='200', padding='5px', width='100%'))